<a href="https://colab.research.google.com/github/Titoffifee/AltGU/blob/main/AGU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost
!pip install pymorphy2
!pip install scipy

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [72]:
import pandas as pd
import numpy as np
import scipy
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from nltk.stem import WordNetLemmatizer

In [73]:
workers = pd.read_csv('/content/drive/MyDrive/Владивосток/AltGU/employees.csv')
train = pd.read_csv('/content/drive/MyDrive/Владивосток/train_and_test_data/train_issues - train_issues.csv')
test = pd.read_csv('/content/drive/MyDrive/Владивосток/train_and_test_data/test_issues - test_issues.csv')

In [74]:
train = train.drop(['id', 'created', 'key', 'project_id', 'summary'], axis=1)
test = test.drop(['id', 'created', 'key', 'project_id', 'summary'], axis=1)

In [75]:
train = train.rename(columns={'Summary':'summary'})
test = test.rename(columns={'translated':'summary'})

In [76]:
ids = workers['id']
workers['position'] = workers['position'].fillna('Web-разработчик')
workers['payment_type'] = workers['payment_type'].fillna('fixed')
test[''] = test['summary'].fillna('NAN')
train['summary'] = train['summary'].fillna('NAN')
d = workers[['active', 'position', 'payment_type']].to_dict()

In [78]:
train

,assignee_id,creator_id,overall_worklogs,summary
0,93,93,1800,"UI tests ""Good Cage"""
1,93,93,7200,"UI tests ""Profile"""
2,93,93,14400,"UI tests of ""Personal account"" section"
3,93,93,900,"UI tests of ""News"" section"
4,93,93,900,"UI tests of the ""Discount and Supplement"" areas"
...,...,...,...,...
9584,1,116,26040,Make a Test To Check Teachers Pages for vn & Th
9585,2,2,16200,Set Up
9586,2,2,3600,Set Up a Graph in Neo4j by Gitlab Dataset
9587,2,2,6300,PARSE A DATASET from Teacherly Slack


In [87]:
patterns = "[!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\]+-0123456789"
table = dict()
for el in patterns:
  table[ord(el)] = ' '
stop = nltk.corpus.stopwords.words("english")
stemmer = WordNetLemmatizer()

def lemmatize(doc):
  doc = doc.translate(table)
  tokens = [stemmer.lemmatize(word).lower() for word in doc.split()]
  tokens = [word for word in tokens if word not in stop]
  return ' '.join(tokens)

test['summary'] = test['summary'].apply(lemmatize)
train['summary'] = train['summary'].apply(lemmatize)

In [ ]:
train

In [89]:
texts = pd.concat([test['summary'], train['summary']])
vectorizer = TfidfVectorizer()
vectorizer.fit(texts)

train_texts = vectorizer.transform(train['summary'])
test_texts = vectorizer.transform(test['summary'])

In [90]:
def GetOneHotDataEncodingData(data1, data2):
  ONE = OneHotEncoder()
  ONE.fit(pd.concat([data1, data2]))
  return (ONE.transform(data1), ONE.transform(data2))


train_assignee, test_assignee = GetOneHotDataEncodingData(train[['assignee_id']], test[['assignee_id']])

In [91]:
X = pd.DataFrame(scipy.sparse.hstack([train_assignee, train_texts]).toarray())
X_test = pd.DataFrame(scipy.sparse.hstack([test_assignee, test_texts]).toarray())
y = train['overall_worklogs']

In [97]:
model = KNeighborsRegressor()
model_for_predict = KNeighborsRegressor(n_neighbors=13, metric='cosine', 
                                        algorithm='brute', weights='uniform')

In [24]:
parameters = {'n_neighbors': list(range(2, 14, 2)),
              'metric': ['cosine', 'minkowski'],
              'algorithm': ['brute', 'auto'], 
              'weights': ['distance', 'uniform']}
Grid = GridSearchCV(estimator=model_knn, param_grid=parameters, cv=5, n_jobs=-1)
Grid.fit(X, y)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'algorithm': ['brute', 'auto'],
                         'metric': ['cosine', 'minkowski'],
                         'n_neighbors': [2, 4, 6, 8, 10, 12],
                         'weights': ['distance', 'uniform']})

In [25]:
Grid.best_params_

{'algorithm': 'brute',
 'metric': 'cosine',
 'n_neighbors': 12,
 'weights': 'uniform'}

In [98]:
from sklearn.utils import shuffle
from sklearn.model_selection import cross_validate

result = cross_validate(model_for_predict, X, y, cv=5, scoring='r2')
print(result['test_score'])
print(result['test_score'].mean())

[-0.14105567  0.06547635 -0.10990927 -5.41506659 -0.83817559]
-1.287746154219482


In [100]:
model_for_predict.fit(X, y)
result = model_for_predict.predict(X_test)
result

array([10315.38461538,  9715.38461538, 26400.        , ...,
        7938.46153846, 12650.76923077,  6221.53846154])

In [ ]:
df_result = pd.read_csv('/content/drive/MyDrive/Владивосток/AltGU/sample_solution.csv')
df_result['overall_worklogs'] = result
df_result.to_csv('/content/drive/MyDrive/Владивосток/ans.csv', index=False)
df_result